## Exact diagonalization of superspin chain quantization of CC network model

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sympy import Matrix

M is the number of bosons in the truncation of bosonic Hilbert space, N is the number of spins in the chain

In [12]:
M = 3
N = 6

Generate matrices that will define the local Hilbert space

In [13]:

Diag1 = np.pad([np.sqrt(i) for i in range(1, M+1)], (0,M))
Diag2 = np.pad([np.sqrt(i) for i in range(1, M)], (M+1,1))
Diag3 = np.pad([np.sqrt(i) for i in range(1, M+1)], (M,0))
Diag4 = np.pad([np.sqrt(i) for i in range(1, M)], (0,M))
bR = np.diag(Diag1, k= -M) + np.diag(Diag2, k= 1-M)
bdgR = np.transpose( bR)
bA = np.diag(Diag3, k = M) + np.diag(Diag4, k = M+1)
bdgA = np.transpose( bA)
NbrR =  bdgR@ bR + np.diag(np.pad(range(M), (2*M, 0)))
NbrA =  bdgA@ bA + np.diag(np.pad(range(M), (0, 2*M)))


bR = np.kron( bR, np.eye(4))
bdgR = np.kron( bdgR, np.eye(4))
bA = np.kron( bA, np.eye(4))
bdgA = np.kron(bdgA, np.eye(4))
NbrR = np.kron(NbrR, np.eye(4))
NbrA = np.kron(NbrA, np.eye(4))

def FermMatrices(norm):
    f = np.array([[0,1], [0,0]])
    fdg = np.transpose(f)
    fA = np.kron(f, np.eye(2))
    fdgA = np.kron(fdg, norm*np.eye(2))
    PauliZ = np.array([[1,0],[0,-1]])
    fR = np.kron(PauliZ, f).astype(int)
    fdgR = np.kron(norm*PauliZ, fdg).astype(int)
    fR = np.kron(np.eye(3*M),  fR).astype(int)
    fdgR = np.kron(np.eye(3*M),  fdgR).astype(int)
    fA = np.kron(np.eye(3*M),  fA).astype(int)
    fdgA = np.kron(np.eye(3*M),  fdgA).astype(int)
    return fR, fdgR, fA, fdgA

fR, fdgR, fA, fdgA = FermMatrices(+1)
fbR, fbdgR, fbA, fbdgA = FermMatrices(-1)
NfR = fdgR@fR
NfA = fdgA@fA
NfbR = -fbdgR@fbR
NfbA = -fbdgA@fbA

### Helper functions/code to visualize the action of operators on local Hilbert space
   - <code>visualizeState(vector)</code> takes a $12M$ dimensional vector and yields the string that represents that state in human readable form
   - <code>visualizeProjectedState(vector)</code> takes a $4M$ dimensional vector that has been projected into the SUSY subspace (with R particles = A particles) and returns the state in human readable form

In [14]:
stateSpace = []
fermionicStates = ["|>", "|Rf>", "|Af>", "|AfRf>"]

for s in (-1,0,1):
    for i in range(M):
        for fState in fermionicStates:
            fewer = "A" if s <= 0 else "R"
            more = "R" if fewer=="A" else "A"
            stateString = "|" + ''.join(sorted((i + np.abs(s))*more + i*fewer)) + ">"
            stateSpace.append(stateString + fState)

def visualizeState(vector):
    # Helper function to visualize state pre projection
    if len(vector) != 12*M:
        print("Incorrect dims")
        return
    

    resultStr = ''
    for i in range(12*M):
        if vector[i] != 0:
            if vector[i] == 1:
                resultStr += stateSpace[i] + " + "
            else:
                resultStr += str(vector[i]) + stateSpace[i] + " + "
                
    return "0" if resultStr == '' else resultStr[:-3]

In [15]:
projector = []
for index, element in enumerate(visualizeState(np.ones(12*M)).split(" + ")):
    row = np.zeros(12*M)
    if element.count("R") == element.count("A"):
        row[index] = 1
    projector.append(row)
projector = np.array(projector)
projector = np.array(Matrix(projector).rref()[0])[:4*M].astype(int)


In [16]:
projectedSpace = []
for i in range(12*M):
    tempState = np.zeros(12*M)
    tempState[i] = 1
    if np.any(projector@tempState):
        projectedSpace.append(visualizeState(tempState))

def visualizeProjectedState(vector):
    resultStr = ''
    for i in range(4*M):
        if vector[i] != 0:
            if vector[i] == 1:
                resultStr += projectedSpace[i] + " + "
            else:
                resultStr += str(vector[i]) + projectedSpace[i] + " + "
    return "0" if resultStr == '' else resultStr[:-3]

### Functions to insert local operators into the total Hilbert space
 - <code>insertOperator(mat, siteIndex)</code> takes a non-SUSY projected matrix (necessarily bosonic) and inserts it at given site
 - <code>stringOperators(siteIndex)</code> returns a matrix with the string of fermionic operators $\prod_{i = 0}^{j-1}(1 - 2n_{f_{A_i}})(1 - 2n_{f_{R_i}})$
 - <code>term(mat, siteIndex, fermionQ)</code> is a catch-all function that simply combines the jobs of both functions above. If <code>fermionQ != 0</code>, then we generate and insert the string operators along with the matrix itself 

In [17]:

def insertOperator(mat, siteIndex):
    # Insert non-SUSY projected bosonic operator on to the site
    if siteIndex > N-1:
        return 'None'
    localDim = 4*M
    projectedMat = projector@mat@np.transpose(projector)
    returnMat = projectedMat if siteIndex == 0 else sparse.identity(localDim)
    for i in range(1,N):
        currentMat = projectedMat if siteIndex == i else sparse.identity(localDim)
        returnMat = sparse.kron(returnMat, currentMat)
    return returnMat

def stringOperators(siteIndex):
    localDim = 12*M
    if siteIndex == 0:
        return sparse.identity((4*M)**N)
    else:
        returnMat = insertOperator((np.eye(localDim) - 2*NfR)@(np.eye(localDim) - 2*NfA), 0)
        for i in range(1, siteIndex):
            RfermCounter = NfR if siteIndex%2 == 0 else NfbR
            AfermCounter = NfA if siteIndex%2 == 0 else NfbA
            returnMat = returnMat@insertOperator((np.eye(localDim) - 2*AfermCounter)@(np.eye(localDim) - 2*RfermCounter), i)
        return returnMat

def term(mat, siteIndex, fermionQ):
    if fermionQ == 0:
        return insertOperator(mat, siteIndex)
    else:
        return stringOperators(siteIndex)@insertOperator(mat, siteIndex)

## Construct the Hamiltonian matrix as per Ilya's notes, equation C.32

In [18]:
eps = 0.01
Hamiltonian = sparse.csr_matrix(((4*M)**N, (4*M)**N))

for i in range(N):
    if i == N-1:
        regSite = 0
        barSite = N-1
        prefactor = -(1 - eps)
    else:
        regSite = i if i%2 == 0 else i+1
        barSite = i+1 if i%2 ==0 else i
        prefactor = -(1 + eps) if i%2 == 0 else -(1 - eps)
    print('J: ', regSite, ' Jbar: ', barSite)
    print('prefactor: ', prefactor)

    
    
    Hamiltonian -= term(NbrR, regSite, 0)@term(NbrR, barSite, 0)
    Hamiltonian -= term(fdgR@bR, regSite, 1)@term(fbdgR@bR, barSite, 1)
    Hamiltonian += term(bR@bA, regSite, 0)@term(bR@bA, barSite, 0)
    Hamiltonian -= term(bR@fA, regSite, 1)@term(bR@fbA, barSite, 1)
    Hamiltonian += term(bdgR@fR, regSite, 1)@term(bdgR@fbR, barSite, 1)
    Hamiltonian += term(NfR, regSite, 0)@term(NfbR, barSite, 0)
    Hamiltonian -= term(fR@bA, regSite, 1)@term(fbR@bA, barSite, 1)
    Hamiltonian += term(fR@fA, regSite, 0)@term(fbR@fbA, barSite, 0)
    Hamiltonian += term(bdgR@bdgA, regSite, 0)@term(bdgR@bdgA, barSite, 0)
    Hamiltonian += term(fdgR@bdgA, regSite, 1)@term(fbdgR@bdgA, barSite, 1)
    Hamiltonian -= term(NbrA, regSite, 0)@term(NbrA, barSite, 0)
    Hamiltonian += term(bdgA@fA, regSite, 1)@term(bdgA@fbA, barSite, 1)
    Hamiltonian += term(bdgR@fdgA, regSite, 1)@term(bdgR@fbdgA, barSite, 1)
    Hamiltonian += term(fdgR@fdgA, regSite, 0)@term(fbdgR@fbdgA, barSite, 0)
    Hamiltonian -= term(fdgA@bA, regSite, 1)@term(fbdgA@bA, barSite, 1)
    Hamiltonian += term(NfA, regSite, 0)@term(NfbA, barSite, 0)
    Hamiltonian -= term(0.5*NbrR, regSite, 0) + term(0.5*NbrA, regSite, 0) + term(0.5*NbrR, barSite, 0) + term(0.5*NbrA, barSite, 0)
    Hamiltonian -= term(0.5*NfR, regSite, 0) + term(0.5*NfA, regSite, 0) + term(0.5*NfbR, barSite, 0) + term(0.5*NfbA, barSite, 0)
    Hamiltonian *= prefactor
    
        

J:  0  Jbar:  1
prefactor:  -1.01
J:  2  Jbar:  1
prefactor:  -0.99
J:  2  Jbar:  3
prefactor:  -1.01
J:  4  Jbar:  3
prefactor:  -0.99
J:  4  Jbar:  5
prefactor:  -1.01
J:  0  Jbar:  5
prefactor:  -0.99


In [20]:
# Check Projections
for i in range(4*M):
    tempState = np.zeros(4*M)
    tempState[i] = 1
    print("State: ", visualizeProjectedState(tempState), "fb anti-commutator :", visualizeProjectedState(projector@(np.eye(12*M)-2*fdgA@fA)@(np.eye(12*M)-2*fdgR@fR).astype(int)@np.transpose(projector)@tempState))



State:  |R>|Af> fb anti-commutator : -1.0|R>|Af>
State:  |ARR>|Af> fb anti-commutator : -1.0|ARR>|Af>
State:  |AARRR>|Af> fb anti-commutator : -1.0|AARRR>|Af>
State:  |>|> fb anti-commutator : |>|>
State:  |>|AfRf> fb anti-commutator : |>|AfRf>
State:  |AR>|> fb anti-commutator : |AR>|>
State:  |AR>|AfRf> fb anti-commutator : |AR>|AfRf>
State:  |AARR>|> fb anti-commutator : |AARR>|>
State:  |AARR>|AfRf> fb anti-commutator : |AARR>|AfRf>
State:  |A>|Rf> fb anti-commutator : -1.0|A>|Rf>
State:  |AAR>|Rf> fb anti-commutator : -1.0|AAR>|Rf>
State:  |AAARR>|Rf> fb anti-commutator : -1.0|AAARR>|Rf>
